In [1]:
import face_recognition
import cv2
import numpy as np 
from os import listdir
from os.path import isfile, join
from glob import glob
import keras
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv3D, MaxPooling3D
from keras import backend as K
import time
import webbrowser

In [29]:
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3),activation='relu', input_shape=(24,100,100,1)))
model.add(Conv3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, (3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.load_weights(r"Python\model\model.h5")

In [30]:
#model.load_weights(r"Python\model\model.h5")

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 22, 98, 98, 32)    896       
                                                                 
 conv3d_5 (Conv3D)           (None, 20, 96, 96, 64)    55360     
                                                                 
 max_pooling3d_3 (MaxPooling  (None, 10, 48, 48, 64)   0         
 3D)                                                             
                                                                 
 conv3d_6 (Conv3D)           (None, 8, 46, 46, 64)     110656    
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 4, 23, 23, 64)    0         
 3D)                                                             
                                                                 
 conv3d_7 (Conv3D)           (None, 2, 21, 21, 64)    

In [32]:
def get_users():

    known_namess=[]
    known_encods=[]
    l="Python\model\*.jpeg"

    for i in glob(l):
        img = face_recognition.load_image_file(i)
        encoding = face_recognition.face_encodings(img)[0]
        known_encods.append(encoding)
        known_namess.append(i[7:-4])

    return known_namess, known_encods

In [33]:
known_names, known_encods = get_users()

In [34]:
known_names

[]

In [35]:
#known_names=[]
#for i in known_names:
    #x=i.split('/')[3]
   # known_names.append(x)

In [36]:
known_encods

[]

In [37]:
font = cv2.FONT_HERSHEY_TRIPLEX

In [38]:
video_capture = cv2.VideoCapture(0)
video_capture.set(3, 640)
video_capture.set(4, 480)

True

In [25]:
video_capture = cv2.VideoCapture(0)
video_capture.set(3, 640)
video_capture.set(4, 480)

face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
input_vid = []

unlock = False

while True:
    # Grab a single frame of video
    if len(input_vid) < 24:

        ret, frame = video_capture.read()

        liveimg = cv2.resize(frame, (100,100))
        liveimg = cv2.cvtColor(liveimg, cv2.COLOR_BGR2GRAY)
        input_vid.append(liveimg)
    else:
        ret, frame = video_capture.read()

        liveimg = cv2.resize(frame, (100,100))
        liveimg = cv2.cvtColor(liveimg, cv2.COLOR_BGR2GRAY)
        input_vid.append(liveimg)
        inp = np.array([input_vid[-24:]])
        inp = inp/255
        inp = inp.reshape(1,24,100,100,1)
        pred = model.predict(inp)
        input_vid = input_vid[-25:]

        if pred[0][0]> .95:

            # Resize frame of video to 1/4 size for faster face recognition processing
            small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

            if process_this_frame:
                # Find all the faces and face encodings in the current frame of video
                face_locations = face_recognition.face_locations(small_frame)
                face_encodings = face_recognition.face_encodings(small_frame, face_locations)
                name = "Unknown"
                face_names = []
                for face_encoding in face_encodings:
                    for ii in range(len(known_encods)):
                        # See if the face is a match for the known face(s)
                        match = face_recognition.compare_faces([known_encods[ii]], face_encoding)

                        if match[0]:
                            name = known_names[ii]

                    face_names.append(name)

            process_this_frame = not process_this_frame

            unlock = False
            for n in face_names:

                if n != 'Unknown':
                    unlock=True


            # Display the results
            for (top, right, bottom, left), name in zip(face_locations, face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                # Draw a box around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with a name below the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)
                if unlock:
                    cv2.putText(frame, 'Verified', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (255, 255, 255), 1)
                else:
                    cv2.putText(frame, 'Not Verified', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (255, 255, 255), 1)
        else:
            cv2.putText(frame, 'WARNING!', (frame.shape[1]//2, frame.shape[0]//2), font, 1.0, (255, 255, 255), 1)
        # Display the liveness score in top left corner     
        cv2.putText(frame, str(pred[0][0]), (10, 10), font, 1.0, (255, 255, 0), 1)
        # Display the resulting image
        cv2.imshow('Video', frame)
                
        
        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        n1='s\\ADMIN\\Python\\model\\Azhar.'
        n2='s\\ADMIN\\Python\\model\\Sheik.'
       # n3='model\\Gokul'

        if (unlock==True):
            if (n1 in face_names):
                time.sleep(3)
                webbrowser.open('https://test-test-123456.my.canva.site/is-this-your')
                break
            elif(n2 in face_names):
                time.sleep(3)
                webbrowser.open('https://test-test-123456.my.canva.site/ka-02-js-1642')
                break
            #elif(n3 in face_names):
               # time.sleep(3)
                #webbrowser.open('https://test-test-123456.my.canva.site/tn-12-cv-7029')
                #break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 1s 633ms/step
